<a href="https://colab.research.google.com/github/fatemafaria142/Impact-of-Instruction-Tuning-in-Biomedical-Language-Processing-Using-Different-LLMs/blob/main/Biomedical_Language_Processing_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━

#### **Dataset Link:** Dataset Link: https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions?row=0

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [3]:
# Select 500 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(500))

# Select 200 rows for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(500, 700))


In [4]:
train_subset

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 500
})

In [5]:
test_subset

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 200
})

In [6]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 500
Test subset size: 200


In [7]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

-----------------------------



#### **Creating a Mental Health Support Prompt Using LangChain**

In [17]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST]<<SYS>> {instruction}\n"""
    prompt = pre_prompt + "{input}" +"[/INST]"+"\n{output}"

    # Extract instruction, input, and output from the sample
    example_instruction = sample['instruction']
    example_input = sample['input']
    example_output = sample['output']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["instruction", "input","output"])
    single_turn_prompt = prompt_template.format(instruction=example_instruction, input=example_input,output=example_output)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [18]:
prompt = create_prompt(train_subset[0])
print(prompt)

[INST]<<SYS>> In your role as a medical professional, address the user's medical questions and concerns.
My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.[/INST]
Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.


In [19]:
prompt = create_prompt(train_subset[1])
print(prompt)

[INST]<<SYS>> Your role as a doctor requires you to answer the medical questions taking into account the patient's description.
Analyze the question given its context. Give both long answer and yes/no decision.
###Question: Are fibrocytes involved in inflammation as well as fibrosis in the pathogenesis of Crohn 's disease?
###Context: We previously showed that fibrocytes, a hematopoietic stem cell source of fibroblasts/myofibroblasts, infiltrated the colonic mucosa of a murine colitis model. We investigated whether fibrocytes were involved in the pathogenesis of Crohn's disease. Human surgical intestinal specimens were stained with anti-leukocyte-specific protein 1 and anti-collagen type-I (ColI) antibodies. Circulating fibrocytes in the human peripheral blood were quantified by fluorescence-activated cell sorting with anti-CD45 and anti-ColI antibodies. Cultured human fibrocytes were prepared by culturing peripheral CD14(+) monocytes. In the specimens of patients with Crohn's disease,

In [20]:
prompt = create_prompt(train_subset[2])
print(prompt)

[INST]<<SYS>> Your identity is a doctor, kindly provide answers to the medical questions with consideration of the patient's description.
Analyze the question and answer with the best option.
###Question: Afterhyperpolarization due to
###Options:
A. Na efflux
B. Na+ influx
C. CI influx
D. K+ efflux
[/INST]
###Rationale: Slow return of the K+ channels to the closed state thus K+ efflux.(Ref: Textbook of physiology AK Jain 5th edition page no.36)

###Answer: OPTION D IS CORRECT.


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [21]:
prompt = create_prompt(test_subset[0])
print(prompt)

[INST]<<SYS>> In the clinical text, your objective is to identify relationships between medical problems, treatments, and tests. Medical problems are tagged as @problem$, medical tests as @test$, and treatments as @treatment$. Classify the relationship between two entities as one of the following:
Treatment improves medical problem (TrIP)
Treatment worsens medical problem (TrWP)
Treatment causes medical problem (TrCP)
Treatment is administered for medical problem (TrAP)
Treatment is not administered because of medical problem (TrNAP)
Test reveals medical problem (TeRP)
Test conducted to investigate medical problem (TeCP)
Medical problem indicates medical problem (PIP)
No Relations
However , a septic component to @problem$ was also considered as the pt was found to have @problem$ and C diff in her stool .[/INST]
No Relations


In [22]:
prompt = create_prompt(test_subset[1])
print(prompt)

[INST]<<SYS>> Your task is to recognize and label Named Entities related to diseases in the text. Use the BIO labeling scheme, marking the first word of a disease-related phrase as B (Begin), and label the subsequent words within that phrase as I (Inner). Words unrelated to diseases should be labeled as O.
Angina and ischemic electrocardiographic changes occurred after administration of oral dipyridamole in four patients awaiting urgent myocardial revascularization procedures .[/INST]
Angina : B
and : O
ischemic : O
electrocardiographic : O
changes : O
occurred : O
after : O
administration : O
of : O
oral : O
dipyridamole : O
in : O
four : O
patients : O
awaiting : O
urgent : O
myocardial : O
revascularization : O
procedures : O
. : O


In [23]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST]<<SYS>> Being a doctor, your task is to answer the medical questions based on the patient's description.
Determine the relation between the two medical terms.
###Question: What is the relation between Inferior vena cavogram (procedure) and CV system ?[/INST]
###Answer: Inferior vena cavogram (procedure) has procedure site: CV system


In [24]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST]<<SYS>> Being a doctor, your task is to answer the medical questions based on the patient's description.
Determine the relation between the two medical terms.
###Question: What is the relation between Inferior vena cavogram (procedure) and CV system ?[/INST]
###Answer: Inferior vena cavogram (procedure) has procedure site: CV system


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [25]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [26]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [27]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [28]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [29]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [30]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.7  # You can adjust this value based on your preference
temperature = 0.5  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id,  top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)

# Creating prompt for large language model
pre_prompt = """[INST]<<SYS>> {instruction}\n"""
prompt = pre_prompt + "{input}" +"[/INST]"

prompt = PromptTemplate(template=prompt, input_variables=["instruction","input"])
chain = prompt | llm

instruction = '''Being a doctor, your task is to answer the medical questions based on the patient's description.
Determine the relation between the two medical terms.'''
input = "What is the relation between Inferior vena cavogram (procedure) and CV system ?"
print(chain.invoke({"instruction":instruction,"input": input}))


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  As a doctor, I can help you understand the relation between Inferior Vena Cavogram (procedure) and the Cardiovascular (CV) system.

Inferior Vena Cavogram (IVC) is a medical procedure that involves inserting a catheter through a vein in the lower abdomen and legs to access the inferior vena cava (IVC), which is a large vein that carries deoxygenated blood from the lower body back to the heart. The purpose of the procedure is to diagnose and treat various conditions such as deep vein thrombosis, pulmonary embolism, and inferior vena cava syndrome.

The CV system, on the other hand, refers to the cardiovascular system of the body, which includes the heart, blood vessels, and blood circulation. The CV system is responsible for delivering oxygenated blood to the body's tissues and organs and removing deoxygenated blood from the body.

The relation between IVC and CV system is that the IVC is a part of the CV system. The IVC is a major vein that drains blood from the lower body and brings

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [31]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [32]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [33]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [34]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [35]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.599200
20,2.090300
30,2.047400
40,1.859800
50,1.691100
60,1.707600
70,1.555600
80,1.383500
90,1.448300
100,1.431700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=150, training_loss=1.663440570831299, metrics={'train_runtime': 1187.1748, 'train_samples_per_second': 0.505, 'train_steps_per_second': 0.126, 'total_flos': 1.2186386694144e+16, 'train_loss': 1.663440570831299, 'epoch': 2.07})

### **Save the model**

In [36]:
trainer.save_model("Llama_instruct_generation")

In [37]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

### **Conversation Buffer memory**

In [53]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nAnalyze the question and answer with the best option.\n"""
prompt = pre_prompt + "Here is my question {context}"+"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["context"])

In [54]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [55]:
context = '''###Question: All of the following provisions are included in the Primary health care according to the Alma Ata declaration except:
###Options:
A. Adequate supply of safe drinking water
B. Nutrition
C. Provision of free medicines
D. Basic sanitation'''

chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Analyze the question and answer with the best option.
Here is my question ###Question: All of the following provisions are included in the Primary health care according to the Alma Ata declaration except:
###Options:
A. Adequate supply of safe drinking water
B. Nutrition
C. Provision of free medicines
D. Basic sanitation[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Great! Let's analyze the question and answer options:\n\nQuestion: All of the following provisions are included in the Primary health care according to the Alma Ata declaration except:\n\nOptions:\nA. Adequate supply of safe drinking water\nB. Nutrition\nC. Provision of free medicines\nD. Basic sanitation\n\nThe Alma Ata declaration is a widely recognized international document that outlines the essential components of primary health care. According to the declaration, primary health care should include the following provisions:\n\n* Adequate supply of safe drinking water (Option A)\n* Nutrition (Option B)\n* Provision of free medicines (Option C)\n* Basic sanitation (Option D)\n\nTherefore, the correct answer is (A) Adequate supply of safe drinking water.\n\nExplanation:\nThe Alma Ata declaration emphasizes the importance of access to safe drinking water as a fundamental aspect of primary health care. The declaration recognizes that safe drinking water is essential for maintaining 